In [68]:
import warnings
warnings.filterwarnings('ignore')

In [69]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [70]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [71]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [72]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [73]:
# columns with only 1 unique value 
df.loc[:, df.nunique() == 1]

,pymnt_plan,recoveries,collection_recovery_fee,policy_code,acc_now_delinq,num_tl_120dpd_2m,num_tl_30dpd,tax_liens,hardship_flag,debt_settlement_flag
0,n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,N,N
1,n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,N,N
2,n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,N,N
3,n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,N,N
4,n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,N,N
...,...,...,...,...,...,...,...,...,...,...
68812,n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,N,N
68813,n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,N,N
68814,n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,N,N
68815,n,0.0,0.0,1.0,0.0,0.0,0.0,0.0,N,N


In [74]:
# dropping columns 
df2 = df.drop(columns=["pymnt_plan", "recoveries", "collection_recovery_fee", "policy_code", "acc_now_delinq", "num_tl_120dpd_2m", "num_tl_30dpd", "tax_liens", "hardship_flag", "debt_settlement_flag"])
df2.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,dti,delinq_2yrs,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,27.24,0.0,...,8.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,20.23,0.0,...,17.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,24.26,0.0,...,8.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,31.44,0.0,...,10.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,18.76,0.0,...,14.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0


In [75]:
# checking types
df2.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 76 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   loan_status                 68817 non-null  object 
 8   dti                         68817 non-null  float64
 9   delinq_2yrs                 68817 non-null  float64
 10  inq_last_6mths              68817 non-null  float64
 11  open_acc                    68817 non-null  float64
 12  pub_rec                     68817 non-null  float64
 13  revol_bal                   688

In [76]:
# convert string vaiables with get dummies 
df_encoded = pd.get_dummies(df2, columns=["home_ownership", "verification_status", "issue_d", "initial_list_status", "next_pymnt_d", "application_type"])
df_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,0,1,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,0,1,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,0,1,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,0,0,1,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,0,0,1,0,1,0,1,1,0


In [77]:
#convert target column to int using label encoder 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_encoded2 = df_encoded.copy()
df_encoded2["loan_status"] = le.fit_transform(df_encoded2["loan_status"])
df_encoded2.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,1,27.24,0.0,0.0,8.0,0.0,...,0,0,0,1,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,1,20.23,0.0,0.0,17.0,1.0,...,1,0,0,1,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,1,24.26,0.0,0.0,8.0,0.0,...,1,0,0,1,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,1,31.44,0.0,1.0,10.0,1.0,...,1,0,0,1,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,1,18.76,0.0,1.0,14.0,0.0,...,0,0,0,1,0,1,0,1,1,0


In [79]:
df_encoded2["loan_status"].unique()

array([1, 0])

# Split the Data into Training and Testing

In [80]:
# Create our features
X = df_encoded.copy()
X = X.drop("loan_status", axis=1)


# Create our target
y = df_encoded["loan_status"].values

In [81]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.148001,0.371696,0.451066,0.177238,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.355104,0.483261,0.497603,0.381873,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [82]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [83]:
from sklearn.model_selection import train_test_split
# using random state so I can run the results again
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [84]:
# scale data
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
credit_data_scaled = data_scaler.fit_transform(df_encoded2)

In [85]:
print(np.mean(credit_data_scaled[:,0]))
print(np.std(credit_data_scaled[:,0]))

9.148042836558536e-17
1.0


In [86]:
X_scaler = data_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [87]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=1)
rf_model = rf_model.fit(X_train_scaled, y_train)
pred = rf_model.predict(X_test_scaled)

In [88]:
# Calculated the balanced accuracy score
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, pred)

0.6605982410126868

In [89]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

array([[   28,    59],
       [   11, 17107]])

In [90]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.72      0.32      1.00      0.44      0.57      0.30        87
   low_risk       1.00      1.00      0.32      1.00      0.57      0.34     17118

avg / total       1.00      1.00      0.33      1.00      0.57      0.34     17205



In [91]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08614815336087911, 'total_rec_prncp'),
 (0.071857509096548, 'last_pymnt_amnt'),
 (0.0669411409664482, 'total_rec_int'),
 (0.06638428994414054, 'total_pymnt'),
 (0.05997492879697617, 'total_pymnt_inv'),
 (0.020981785754789838, 'out_prncp'),
 (0.020483863108472188, 'installment'),
 (0.01811387585580564, 'out_prncp_inv'),
 (0.017177915977250203, 'dti'),
 (0.016372153190334603, 'mo_sin_old_rev_tl_op'),
 (0.016074716682474267, 'loan_amnt'),
 (0.015393659594864032, 'total_rec_late_fee'),
 (0.015257124762512975, 'mo_sin_old_il_acct'),
 (0.014749956663084143, 'max_bal_bc'),
 (0.014388712014302665, 'bc_open_to_buy'),
 (0.014283237238161488, 'bc_util'),
 (0.013590632995791496, 'tot_cur_bal'),
 (0.013437487622441171, 'annual_inc'),
 (0.013321821921562648, 'total_bal_ex_mort'),
 (0.012850536595417352, 'revol_bal'),
 (0.012466229628748868, 'total_bc_limit'),
 (0.012318962885062176, 'total_bal_il'),
 (0.012051399246068129, 'tot_hi_cred_lim'),
 (0.012045473637780283, 'total_il_high_credit_limit')

### Easy Ensemble AdaBoost Classifier

In [92]:
# Train the EasyEnsembleClassifier
from sklearn.ensemble import AdaBoostClassifier
classifier = AdaBoostClassifier(n_estimators=100, random_state=0)
classifier.fit(X_train_scaled, y_train)

pred = classifier.predict(X_test_scaled)


In [93]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, pred)

0.6894799182953213

In [94]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

array([[   33,    54],
       [    6, 17112]])

In [95]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.38      1.00      0.52      0.62      0.36        87
   low_risk       1.00      1.00      0.38      1.00      0.62      0.40     17118

avg / total       1.00      1.00      0.38      1.00      0.62      0.40     17205

